### Import libraries and data

In [36]:
# Import libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import plotly.express as px
import contextily as ctx
import plotly.io as pio
import statistics
import plotly.graph_objects as go

In [2]:
# Set working dir & get data
cwd = os.getcwd()
print('Current Working Directory is: ', cwd)

Current Working Directory is:  /Users/dominiqueong/Documents/CCED


In [3]:
# Import block group geojson file from LA County GIS Hub
# Read in 2020 block group geojson file
blockgroups_path = os.path.join("GeoJSON","Census_Block_Groups_2020.geojson")
blockgroups = gpd.read_file(blockgroups_path,dtype={'BG20':str})
# Read in Chinatown neighborhood boundary geojson file
neighborhood_path = os.path.join("LA_Times_Neighborhood_Boundaries","Chinatown_LA_Times_Neighborhood_Boundaries.geojson")
neighborhood = gpd.read_file(neighborhood_path)

In [4]:
# Import Social Explorer ACS 5-Year Estimates CSVs for 2016 and 2021
# Join components of relative file path to output file location
acs_2011_path = os.path.join("Social Explorer","ACS 2011 5-Year Estimates_R13294525_SL150.csv")
acs_2016_path = os.path.join("Social Explorer","ACS 2016 5-Year Estimates_R13293532_SL150.csv")
acs_2021_path = os.path.join("Social Explorer","ACS 2021 5-Year Estimates_R13293531_SL150.csv")
# Read ACS 2016 5-Year Estimates CSV, save data frame
acs_2011 = pd.read_csv(acs_2011_path,dtype={'Geo_FIPS':str,'Geo_TRACT':str,'Geo_BLKGRP':str})
acs_2016 = pd.read_csv(acs_2016_path,dtype={'Geo_FIPS':str,'Geo_TRACT':str,'Geo_BLKGRP':str})
acs_2021 = pd.read_csv(acs_2021_path,dtype={'Geo_FIPS':str,'Geo_TRACT':str,'Geo_BLKGRP':str})
# Get column names and data types for each data frame
acs_2011.info()
acs_2016.info()
acs_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6425 entries, 0 to 6424
Columns: 134 entries, Geo_FIPS to SE_A18007_007
dtypes: float64(83), int64(43), object(8)
memory usage: 6.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6425 entries, 0 to 6424
Columns: 193 entries, Geo_FIPS to SE_A10039B_023
dtypes: float64(97), int64(88), object(8)
memory usage: 9.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6591 entries, 0 to 6590
Columns: 177 entries, Geo_FILEID to SE_A10047_004
dtypes: float64(114), int64(55), object(8)
memory usage: 8.9+ MB


### Trim and rename columns in ACS 5-Year Estimates

In [5]:
# Define columns to keep from ACS CSVs, using existing column names
# SE_A18009_001 = Median Gross Rent, Renter-occupied housing units paying cash rent
# SE_B18002_001 = Renter-Occupied Housing Units
# SE_B18002_002 = Residents Paying More Than 30% of Income on Rent, Renter-occupied housing units
# SE_B18002_003 = Residents Paying at least 50% of Income on Rent, Renter-occupied housing units
# SE_A18001_001 = Renter-Occupied Housing Units with Cash Rent
# SE_A18001_002 = Gross Rent Less than $300, Renter-occupied housing units with cash rent
# SE_A18001_003 = Gross Rent $300 to $599, Renter-occupied housing units with cash rent
# SE_A18001_004 = Gross Rent $600 to $799, Renter-occupied housing units with cash rent
# SE_A18001_005 = Gross Rent $800 to $999, Renter-occupied housing units with cash rent
# SE_A18001_006 = Gross Rent $1,000 to $1,249, Renter-occupied housing units with cash rent
# SE_A18001_007 = Gross Rent $1,250 to $1,499, Renter-occupied housing units with cash rent
# SE_A18001_008 = Gross Rent $1,500 to $1,999, Renter-occupied housing units with cash rent
# SE_A18001_009 = Gross Rent $2,000 or More, Renter-occupied housing units with cash rent
columns_to_keep = ['Geo_FIPS','Geo_TRACT','Geo_BLKGRP','SE_A18009_001','SE_A18001_001','SE_A18001_002','SE_A18001_003','SE_A18001_004','SE_A18001_005','SE_A18001_006','SE_A18001_007','SE_A18001_008','SE_A18001_009']

In [6]:
# Select desired columns from 2011, 2016, and 2021 ACS 5-Year Estimates, save as new dataframes
acs_2011_trim = acs_2011[columns_to_keep].copy()
acs_2016_trim = acs_2016[columns_to_keep].copy()
acs_2021_trim = acs_2021[columns_to_keep].copy()
# Rename columns of 2011, 2016, and 2021 ACS 5-Year Estimates
acs_2011_trim.columns = ['Geo_FIPS','Geo_TRACT','Geo_BLKGRP','Median Gross Rent','Renter-Occupied Housing Units with Cash Rent','Gross Rent Less than $300','Gross Rent $300 to $599','Gross Rent $600 to $799','Gross Rent $800 to $999','Gross Rent $1,000 to $1,249','Gross Rent $1,250 to $1,499','Gross Rent $1,500 to $1,999','Gross Rent $2,000 or More']
acs_2016_trim.columns = ['Geo_FIPS','Geo_TRACT','Geo_BLKGRP','Median Gross Rent','Renter-Occupied Housing Units with Cash Rent','Gross Rent Less than $300','Gross Rent $300 to $599','Gross Rent $600 to $799','Gross Rent $800 to $999','Gross Rent $1,000 to $1,249','Gross Rent $1,250 to $1,499','Gross Rent $1,500 to $1,999','Gross Rent $2,000 or More']
acs_2021_trim.columns = ['Geo_FIPS','Geo_TRACT','Geo_BLKGRP','Median Gross Rent','Renter-Occupied Housing Units with Cash Rent','Gross Rent Less than $300','Gross Rent $300 to $599','Gross Rent $600 to $799','Gross Rent $800 to $999','Gross Rent $1,000 to $1,249','Gross Rent $1,250 to $1,499','Gross Rent $1,500 to $1,999','Gross Rent $2,000 or More']

### Prepare ACS 5-Year Estimates for attribute join (merge)

#### Merge each ACS 5-Year Estimates dataframe to block group geojson file, joining by FIPS column attribute

In [7]:
# Create a FIPS column by appending block group number to state code 06/CA + 037/LA County
blockgroups['FIPS'] = '06' + '037' + blockgroups['BG20']
acs_2011_trim['FIPS'] = '06' + '037' + acs_2011_trim['Geo_TRACT'] + acs_2011_trim['Geo_BLKGRP']
acs_2016_trim['FIPS'] = '06' + '037' + acs_2016_trim['Geo_TRACT'] + acs_2016_trim['Geo_BLKGRP']
acs_2021_trim['FIPS'] = '06' + '037' + acs_2021_trim['Geo_TRACT'] + acs_2021_trim['Geo_BLKGRP']

In [8]:
# Merge each dataframe to block group geojson
acs_2011_rent_blockgroups = blockgroups.merge(acs_2011_trim,on='FIPS')
acs_2016_rent_blockgroups = blockgroups.merge(acs_2016_trim,on='FIPS')
acs_2021_rent_blockgroups = blockgroups.merge(acs_2021_trim,on='FIPS')

### Clip block groups to Chinatown neighborhood boundary

In [9]:
# Clip each year of merged block groups to Chinatown neighborhood boundary
acs_2011_rent_blockgroups_clip = gpd.clip(acs_2011_rent_blockgroups,neighborhood)
acs_2016_rent_blockgroups_clip = gpd.clip(acs_2016_rent_blockgroups,neighborhood)
acs_2021_rent_blockgroups_clip = gpd.clip(acs_2021_rent_blockgroups,neighborhood)

### Calculate number of renter-occupied housing units in each gross rent category for 2016 and 2021

#### Create a dataframe for gross rent categories

In [10]:
# Create a dataframe for gross rent categories
# Less than $300, $300 to $599, $600 to $799, $800 to $999, $1,000 to $1,249, $1,250 to $1,499, $1,500 to $1,999, $2,000 or More
# Define columns with gross rent categories to keep
rent_cat_columns_to_keep = ['Gross Rent Less than $300','Gross Rent $300 to $599','Gross Rent $600 to $799','Gross Rent $800 to $999','Gross Rent $1,000 to $1,249','Gross Rent $1,250 to $1,499','Gross Rent $1,500 to $1,999','Gross Rent $2,000 or More']
# Calculate sum of all values in each gross rent category column for each year
# 2011
acs_2011_rent_cat_sum_chinatown = acs_2011_rent_blockgroups_clip[rent_cat_columns_to_keep].sum(axis=0)
acs_2011_rent_cat_sum_chinatown = pd.DataFrame(acs_2011_rent_cat_sum_chinatown).reset_index()
acs_2011_rent_cat_sum_chinatown['year'] = str('2011-2015')
acs_2011_rent_cat_sum_chinatown.columns = ['category', 'sum','year']
# 2016
acs_2016_rent_cat_sum_chinatown = acs_2016_rent_blockgroups_clip[rent_cat_columns_to_keep].sum(axis=0)
acs_2016_rent_cat_sum_chinatown = pd.DataFrame(acs_2016_rent_cat_sum_chinatown).reset_index()
acs_2016_rent_cat_sum_chinatown['year'] = str('2012-2016')
acs_2016_rent_cat_sum_chinatown.columns = ['category', 'sum','year']
# 2021
acs_2021_rent_cat_sum_chinatown = acs_2021_rent_blockgroups_clip[rent_cat_columns_to_keep].sum(axis=0)
acs_2021_rent_cat_sum_chinatown = pd.DataFrame(acs_2021_rent_cat_sum_chinatown).reset_index()
acs_2021_rent_cat_sum_chinatown['year'] = str('2017-2021')
acs_2021_rent_cat_sum_chinatown.columns = ['category','sum','year']
# Combine multiple data frames into one data frame
cat_label = ['Less than $300','$300 to $599','$600 to $799','$800 to $999','$1,000 to $1,249','$1,250 to $1,499','$1,500 to $1,999','$2,000 or more']
acs_2011_2021_rent_cat_sum_chinatown = pd.concat([acs_2011_rent_cat_sum_chinatown,acs_2016_rent_cat_sum_chinatown,acs_2021_rent_cat_sum_chinatown])
acs_2011_2021_rent_cat_sum_chinatown.info()
    

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 7
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  24 non-null     object
 1   sum       24 non-null     int64 
 2   year      24 non-null     object
dtypes: int64(1), object(2)
memory usage: 768.0+ bytes


#### Create bar chart to show distribution of renter-occupied units per gross rent categories, for each year

In [51]:
fig = px.bar(acs_2011_2021_rent_cat_sum_chinatown, x="year", y="sum", color="category", title="Gross Rent in Chinatown Block Groups",text="category")
fig.show()
fig.write_image("Gross_Rent_Chinatown.pdf")

### Map clipped block groups in Chinatown by gross median rent in 2011,2016,2021

In [13]:
# 2011
acs_2011_med_rent = acs_2011_rent_blockgroups_clip['Median Gross Rent']
# 2016
acs_2016_med_rent = acs_2016_rent_blockgroups_clip['Median Gross Rent']
# 2021
acs_2021_med_rent = acs_2021_rent_blockgroups_clip['Median Gross Rent']

In [14]:
### Count number of rent stabilized units

In [37]:
# Import RSO units from 2016-2022
# Read in RSO block group geojson files
# 2016
rso_2016_path = os.path.join("RSO","rso_2016_shapes_clip.geojson")
rso_2016 = gpd.read_file(rso_2016_path)
# 2017
rso_2017_path = os.path.join("RSO","rso_2017_shapes_clip.geojson")
rso_2017 = gpd.read_file(rso_2017_path)
# 2018
rso_2018_path = os.path.join("RSO","rso_2018_shapes_clip.geojson")
rso_2018 = gpd.read_file(rso_2018_path)
# 2019
rso_2019_path = os.path.join("RSO","rso_2019_shapes_clip.geojson")
rso_2019 = gpd.read_file(rso_2019_path)
# 2020
rso_2020_path = os.path.join("RSO","rso_2020_shapes_clip.geojson")
rso_2020 = gpd.read_file(rso_2020_path)
# 2021
rso_2021_path = os.path.join("RSO","rso_2021_shapes_clip.geojson")
rso_2021 = gpd.read_file(rso_2021_path)
# 2022
rso_2022_path = os.path.join("RSO","rso_2022_shapes_clip.geojson")
rso_2022 = gpd.read_file(rso_2022_path)
rso_2016_2022_list = [rso_2016['RSO Units'].sum(),rso_2017['RSO Units'].sum(),rso_2018['RSO Units'].sum(),rso_2019['RSO Units'].sum(),rso_2020['RSO Units'].sum(),rso_2021['RSO Units'].sum(), rso_2022['RSO Units'].sum()]
rso_years = [2016,2017,2018,2019,2020,2021,2022]
rso_2016_2022_df = pd.DataFrame(rso_2016_2022_list, index = rso_years,columns=['Total RSO Units'])

In [52]:
fig = px.line(rso_2016_2022_df, x=rso_2016_2022_df.index, y="Total RSO Units",text="Total RSO Units",title="Total Number of Rent Stabilized Units in Chinatown, 2016-2022")
fig.update_traces(textposition="top right")
fig.show()
fig.write_image("Chinatown_RSO.pdf")